# Rukovanje podatcima

Na laboratorijskoj vježbi pokazat ćemo neke od osnovnih manipulacija podataka koje se često koriste u praksi. U tu svrhu koristit ćemo skup podataka Titanic jer je prikladan za demonstraciju mnogih transformacija podataka. Koristit ćemo algoritam slučajnih šuma za klasifikaciju podataka kako bismo mogli pratiti kakav utjecaj određene manipulacije podataka imaju na klasifikacijski model.  

In [970]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [923]:
# Budite jako oprezni s korištenjem ove postavke! Ne preporuča se početnicima!
import warnings
warnings.filterwarnings('ignore')

### Učitavanje podataka

In [924]:
# ucitavanje podataka
X = pd.read_csv("titanic.csv")

### Početni pregled podataka

In [925]:
X.shape

(891, 12)

In [926]:
# prikaz skupa za treniranje
X.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [927]:
X.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

Pojašnjenja značajki:
 - Survived - ciljani razred - oznaka je li osoba preživjela ili ne
 - Pclass - razred karte (1., 2. ili 3.)
 - Name - Ime putnika
 - Sex - spol putnika
 - Age - starost putnika u godinama
 - SibSp - broj braće, sestara i/ili supružnika ukrcanih na Titanik
 - Parch - broj roditelja/djece ukrcanih na Titanik
 - Ticket - broj karte
 - Fare - putnička karta
 - Cabin - oznaka kabine
 - Embarked - luka ukrcavanja (C = Cherbourg, Q = Queenstown, S = Southampton)

In [928]:
X.describe()

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

### Monotoni atributi
Provjerimo postoje li monotoni atributi, odnosno atributi čija vrijednost jednoliko raste.

In [929]:
X.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

Funkcija nunique broji jedinstvene vrijednosti po stupcima i može služiti kao dobar indikator monotonih atributa. U ovom slučaju kandidati za monotoni atribut su PassengerId i Name jer svaki zapis ima jedinstvenu vrijednost. Lako je zaključiti da je PassengerId monotoni atribut, a Name nije monotoni atribut. Izbacimo atribut PassengerId iz skupa podataka.

In [930]:
X.drop(('PassengerId'), axis=1, inplace=True)

### Klasifikacija #1

Pokušamo li napraviti klasifikaciju koristeći trenutnu verziju podataka, dogodit će se pogreška zato što algoritmi iz modula sklearn rade isključivo s numeričkim vrijednostima. 

In [931]:
# definicija funkcije koju ćemo koristiti za klasifikaciju kroz cijelu bilježnicu
def klasificiraj(df):
    # odvajanje oznake klase
    X = df.loc[:, df.columns != 'Survived']
    y = df.loc[:, 'Survived']
    
    ######## Ovako izgleda treniranje modela na pojedinačnoj podijeli podataka ########
    
    # razdvoji podatke
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    # instanciraj i treniraj model 
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    
    ######## Preporučeni način validiranja modela je korištenjem unakrsne provjere ########
    # unakrsna valicadija
    scores = cross_val_score(model, X, y, cv=20)
    
    
    # evaluiraj model
    print('Točnost: ', scores.mean(), ' +- ', scores.std())
   
# pozovi definiranu funkciju
klasificiraj(X)

ValueError: could not convert string to float: 'McCarthy, Mr. Timothy J'

Pogreška se dogodila zato što algoritmi iz modula sklearn rade isključivo s numeričkim vrijednostima. Pogledajmo koje su sve vrijednosti numeričke.

In [932]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [933]:
# napravimo klasifikaciju isključivo s numeričkim značajkama
X_tmp = X.loc[:,['Survived','Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
klasificiraj(X_tmp)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

### Nedostajući podatci

Sada imamo novi problem s podatcima - nedostajući podatci. Provjerimo koliko je takvih podataka.

In [934]:
X.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

Generalno, opcije za rješavanje problema nedostajućih vrijednosti su:
 - zamijeniti srednjom vrijednošću
 - u potpunosti izbaciti te zapise iz skupa podataka
 - u potpunosti izbaciti te značajke iz skupa podataka
 - interpolacija
 - pronalazak točnih vrijednosti iz drugih izvora podataka
 - ...
 
U našem slučaju značajke godina (Age): 
 - nedostajuće godine zamijenit ćemo srednjom vrijednošću
 - izbacivanje tih zapisa nije opcija jer gubimo previše podataka (gotovo 20%)
 - izbacivanje te značajke nije opcija jer je značajka previše bitna - djecu (i žene) se često prve spašava
 - interpolacija u ovom slučaju nema nikakvog smisla
 - pronalazak točnih vrijednosti iz drugih izvora podataka je najbolja opcija, ali je mi ovdje nećemo raditi radi jednostavnosti vježbe
 
U slučaju značajke kabine (Cabin):
 - nije numerička vrijednost tako da srednja vrijednost nije ocpija
 - izbacivanje tih zapisa nije opcije jer gubimo previše podataka (77%)
 - izbacivanje te značajke je ono što ćemo ovdje napraviti
 - interpolacija nema smisla jer se ne radi o numeričkoj vrijednosti
 - pronalazak točnih vrijednosti iz drugih izvora podataka je najbolja opcija, ali je mi ovdje nećemo raditi radi jednostavnosti vježbe
 
U slučaju značajke luka ukrcavanja (Embarked):
 - nije numerička vrijednost tako da srednja vrijednost nije ocpija 
 - izbacivanje tih zapisa je ono što ćemo ovdje napraviti
 - izbacivanje te značajke nema smisla jer gubimo cijelu značajku zbog samo dva primjera s nedostajućom vrijednosti
 - interpolacija nema smisla jer se ne radi o numeričkoj vrijednosti
 - pronalazak točnih vrijednosti iz drugih izvora podataka je najbolja opcija, ali je mi ovdje nećemo raditi radi jednostavnosti vježbe

In [935]:
X_tmp = X.copy()

# ubaci srednju vrijednost godina tamo gdje nedostaje ta informacija
X_tmp.loc[X_tmp.Age.isna(),'Age'] = X_tmp.loc[:,'Age'].mean()

# izbaci stupac Cabine
X_tmp.drop(['Cabin'], axis=1, inplace=True)

# izbaci zapise s nedostajućom vrijednosti značajke Embarked
X_tmp = X_tmp.loc[X_tmp.Embarked.notnull(), :]

In [936]:
# napravimo klasifikaciju isključivo s numeričkim značajkama
X_tmp = X_tmp.loc[:,['Survived','Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
klasificiraj(X_tmp)

Točnost:  0.6886363636363637  +-  0.058314964968449595


Uspjeli smo napraviti prvu klasifikaciju s točnošću od 68.8%. Pozabavimo se dalje podatcima da vidimo možemo li poboljšati taj rezultat.

### Stršeći podatci

Provjerimo ima li u skupu podataka stršećih vrijednosti.

In [937]:
X_tmp.describe()

Survived      Pclass         Age       SibSp       Parch        Fare
count  889.000000  889.000000  889.000000  889.000000  889.000000  889.000000
mean     0.382452    2.311586   29.653446    0.524184    0.382452   32.096681
std      0.486260    0.834700   12.968366    1.103705    0.806761   49.697504
min      0.000000    1.000000    0.420000    0.000000    0.000000    0.000000
25%      0.000000    2.000000   22.000000    0.000000    0.000000    7.895800
50%      0.000000    3.000000   29.699118    0.000000    0.000000   14.454200
75%      1.000000    3.000000   35.000000    1.000000    0.000000   31.000000
max      1.000000    3.000000   80.000000    8.000000    6.000000  512.329200

Temeljem ovog ispisa kandidati za stršeće podatke su SibSp, Parch i Fare. Pogledamo li SibSp i Parch, maksimalne vrijednosti dosta odskaču, ali nisu nemoguće. Pogledajmo dodatno značajku Fare.

In [938]:
print('Fare > 100: ', (X_tmp.Fare > 100).sum())
print('Fare > 200: ', (X_tmp.Fare > 200).sum())
print('Fare > 300: ', (X_tmp.Fare > 300).sum())
print('Fare > 400: ', (X_tmp.Fare > 400).sum())
print('Fare > 500: ', (X_tmp.Fare > 500).sum())

Fare > 100:  53
Fare > 200:  20
Fare > 300:  3
Fare > 400:  3
Fare > 500:  3


In [939]:
X_tmp.loc[X_tmp.Fare > 500, 'Fare']

258    512.3292
679    512.3292
737    512.3292
Name: Fare, dtype: float64

Ova tri zapisa jako odudaraju od ostalih zapisa i mogu se smatrati stršećim vrijednostima. Za sada nećemo ništa poduzimati po tom pitanju, no te činjenice treba biti svjestan prilikom modeliranja.  
NAPOMENA: Dobar način detekcije stršećih vrijednosti su vizualizacije, a o njima će biti više govora na sljedećem labosu.

### Nekonzistentni podatci
Bilo kakav slobodan unos teksta često dovodi do nekonzistentnosti u podatcima. Primjerice, titula Ms je sinonimom za titulu Miss. Provjerimo koje sve titule postoje u našem skupu podataka (koristeći regularni izraz) i kojem spolu pripadaju.

In [940]:
# stvorimo novi stupac Title korištenjem raularnog izraza
X['Title'] = X.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
# ispišimo učestalost novog stupca po spolovima
pd.crosstab(X['Title'], X['Sex'])

Sex       female  male
Title                 
Capt           0     1
Col            0     2
Countess       1     0
Don            0     1
Dr             1     6
Jonkheer       0     1
Lady           1     0
Major          0     2
Master         0    40
Miss         182     0
Mlle           2     0
Mme            1     0
Mr             0   517
Mrs          125     0
Ms             1     0
Rev            0     6
Sir            0     1

Čini se kako u ovom skupu podataka nema velikih nekonzistentnosti po pitanju unosa titula putnika. Titule Mlle i Ms prebacit ćemo u grupu Miss (ukupno 3 zapisa) i titulu Mme u grupu Mrs (1 zapis).  
Titule koji se rijetko pojavljuju možemo sve grupirati u jednu grupu koju ćemo nazvati Rare.

In [941]:
# stovri titulu Rare
X['Title'] = X['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
# zamijeni nekonzistentne titule
X['Title'] = X['Title'].replace('Mlle', 'Miss')
X['Title'] = X['Title'].replace('Ms', 'Miss')
X['Title'] = X['Title'].replace('Mme', 'Mrs')

Prikažimo postotak preživljavanja u ovisnosti o tituli putnika.

In [942]:
X[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

Title  Survived
0  Master  0.575000
1    Miss  0.702703
2      Mr  0.156673
3     Mrs  0.793651
4    Rare  0.347826

Vidimo kako titula ima velik utjecaj na šansu za preživljavanje, što titulu čini bitnom značajkom za predikciju. Titula je trenutno značajka u obliku teksta. Transformirajmo titulu u numeričku vrijednost kako bismo je mogli koristiti prilikom klasifikacije.

### Rijetki podatci - OneHotEncoder
Titulu kao značajku transformirat ćemo koristeći OneHotEncoder, što će rezultirati s pet novih stupaca (značajki). Za nove stupce vrijedi da će svaki zapis imati samo u jednom od tih pet stupaca vrijednost 1, a u preostala četiri stupca imat će vrijednost 0. Pogledajte primjer niže.

In [943]:
# definiramo enkoder
encoder = OneHotEncoder(handle_unknown="ignore")
encoder.fit(X[['Title']])
# transformiramo podatke
X_tmp = encoder.transform(X[['Title']])
X_tmp = pd.DataFrame(X_tmp.toarray(), columns=encoder.get_feature_names(['Title']))
X_tmp

Title_Master  Title_Miss  Title_Mr  Title_Mrs  Title_Rare
0             0.0         0.0       1.0        0.0         0.0
1             0.0         0.0       0.0        1.0         0.0
2             0.0         1.0       0.0        0.0         0.0
3             0.0         0.0       0.0        1.0         0.0
4             0.0         0.0       1.0        0.0         0.0
..            ...         ...       ...        ...         ...
886           0.0         0.0       0.0        0.0         1.0
887           0.0         1.0       0.0        0.0         0.0
888           0.0         1.0       0.0        0.0         0.0
889           0.0         0.0       1.0        0.0         0.0
890           0.0         0.0       1.0        0.0         0.0

[891 rows x 5 columns]

Spojimo sada dobivenu rijetku tablicu s našom izvornom tablicom i provjerimo popravljaju li nove značajke točnost predikcije.

In [944]:
# spoji podatke
X = pd.concat((X, X_tmp), axis=1)

# izbaci nenumeričke stupce - ~ u ovom slučaju označava "not", odnosno uvjet bi se čitao kao "columns not in ['Name', ...]"
X_tmp = X.loc[:, ~X.columns.isin(['Name','Sex', 'Ticket', 'Cabin', 'Embarked', 'Title'])].copy()

# ponovno ubacimo srednju vrijednost godina jer smo je ranije izračunali samo u privremeni DataFrame
X_tmp.loc[X_tmp.Age.isna(),'Age'] = X_tmp.loc[:,'Age'].mean()

# klasificiraj
klasificiraj(X_tmp)

Točnost:  0.8182828282828283  +-  0.0684515055449198


Vidimo kako je ubacivanje titule kao rijetke značajke rezultiralo porastom točnosti modela s 68.8% na 81.8%.

### LabelEncoder
Do sada nismo koristili značajku spola jer je u tekstualnom formatu. Promijenimo format značajke koristeči LabelEncoder i ponovimo klasifikaciju.

In [945]:
# inicijalizacija enkodera
le = LabelEncoder()
le.fit(X.loc[:,'Sex'])

X.loc[:,'Sex'] = le.transform(X.loc[:,'Sex'])

In [946]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Survived      891 non-null    int64  
 1   Pclass        891 non-null    int64  
 2   Name          891 non-null    object 
 3   Sex           891 non-null    int32  
 4   Age           714 non-null    float64
 5   SibSp         891 non-null    int64  
 6   Parch         891 non-null    int64  
 7   Ticket        891 non-null    object 
 8   Fare          891 non-null    float64
 9   Cabin         204 non-null    object 
 10  Embarked      889 non-null    object 
 11  Title         891 non-null    object 
 12  Title_Master  891 non-null    float64
 13  Title_Miss    891 non-null    float64
 14  Title_Mr      891 non-null    float64
 15  Title_Mrs     891 non-null    float64
 16  Title_Rare    891 non-null    float64
dtypes: float64(7), int32(1), int64(4), object(5)
memory usage: 115.0+ KB


In [947]:
# izbaci nenumeričke stupce - ~ u ovom slučaju označava "not", odnosno uvjet bi se čitao kao "columns not in ['Name', ...]"
X_tmp = X.loc[:, ~X.columns.isin(['Name', 'Ticket', 'Cabin', 'Embarked', 'Title'])].copy()

# ponovno ubacimo srednju vrijednost godina jer smo je ranije izračunali samo u privremeni DataFrame
X_tmp.loc[X_tmp.Age.isna(),'Age'] = X_tmp.loc[:,'Age'].mean()
    
# klasificiraj
klasificiraj(X_tmp)

Točnost:  0.8261363636363637  +-  0.06595268789138127


Dodavanje spola kao značajke povećalo je točnost modela s 81.8% na 82.6%. To je bilo i očekivano jer je poznato da su se žene prve ukrcavale na čamce za spašavanje. Provjerimo postotak preživljavanja u ovisnosti o spolu.

In [948]:
X[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean()

Sex  Survived
0    0  0.742038
1    1  0.188908

Vidimo kako spol jako dobro razdvaja preživjele od preminulih.  
  
Napravimo istu stvar s značajkom Embarked.

In [949]:
# izbaci zapise s nedostajućom vrijednosti značajke Embarked
X = X.loc[X.Embarked.notnull(), :]

# inicijalizacija enkodera
le = LabelEncoder()
le.fit(X.loc[:,'Embarked'])

# transformiraj 
X.loc[:,'Embarked'] = le.transform(X.loc[:,'Embarked'])

In [950]:
X[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean()

Embarked  Survived
0         0  0.553571
1         1  0.389610
2         2  0.336957

In [951]:
# izbaci nenumeričke stupce - ~ u ovom slučaju označava "not", odnosno uvjet bi se čitao kao "columns not in ['Name', ...]"
X_tmp = X.loc[:, ~X.columns.isin(['Name', 'Ticket', 'Cabin', 'Title'])].copy()

# ponovno ubacimo srednju vrijednost godina jer smo je ranije izračunali samo u privremeni DataFrame
X_tmp.loc[X_tmp.Age.isna(),'Age'] = X_tmp.loc[:,'Age'].mean()
    
# klasificiraj
klasificiraj(X_tmp)

Točnost:  0.8133585858585859  +-  0.05677191719127784


Sada imamo smanjenje točnosti iako vjerojatnost preživljavanja ovisi o značajci Embarked. Unatoč tome, značajku Embarked emoostavit ćemo u skupu podataka. Za sada nam je cilj kreirati čim više dobrih značajki, a u kasnijim fazama se može provoditi selekcija značajki ako se za time pokaže potreba.

Razmislimo ponovno o načinu na koji smo popunili nedostajuće vrijednosti za značajku dobi. Sada kada imamo lako dostupan podatak o tituli osobe, možemo pokušati pametnije popuniti nedostajuće vrijednosti dobi. Primjerice, očekivano je da će prosječna dob osobe s titulom Miss biti manja od prosječne dobi osobne s titulom Mrs. Provjerimo.

In [952]:
title_age = X[['Title', 'Age']].groupby(['Title'], as_index=False).mean()
title_age

Title        Age
0  Master   4.574167
1    Miss  21.736486
2      Mr  32.368090
3     Mrs  35.546296
4    Rare  45.545455

Popunimo nedostajuće vrijednosti dobi u odnosu na titulu putnika te provjerimo dovodi li to do dodatnog poboljšanja točnosti modela.

In [953]:
# izbaci nenumeričke stupce - ~ u ovom slučaju označava "not", odnosno uvjet bi se čitao kao "columns not in ['Name', ...]"
X_tmp = X.loc[:, ~X.columns.isin(['PassengerId', 'Name','Sex', 'Ticket', 'Cabin', 'Embarked', 'Title'])].copy()

# popunjavanje nedostajućih vrijednosti
def popuni_nedostajucu_dob(X_tmp):
    mask = (X_tmp.Title_Master == 1) & (X_tmp.Age.isna())
    X_tmp.loc[mask, 'Age'] = title_age.loc[title_age.Title == 'Master', 'Age'].values[0]
    mask = (X_tmp.Title_Miss == 1) & (X_tmp.Age.isna())
    X_tmp.loc[mask, 'Age'] = title_age.loc[title_age.Title == 'Miss', 'Age'].values[0]
    mask = (X_tmp.Title_Mr == 1) & (X_tmp.Age.isna())
    X_tmp.loc[mask, 'Age'] = title_age.loc[title_age.Title == 'Mr', 'Age'].values[0]
    mask = (X_tmp.Title_Mrs == 1) & (X_tmp.Age.isna())
    X_tmp.loc[mask, 'Age'] = title_age.loc[title_age.Title == 'Mrs', 'Age'].values[0]
    mask = (X_tmp.Title_Rare == 1) & (X_tmp.Age.isna())
    X_tmp.loc[mask, 'Age'] = title_age.loc[title_age.Title == 'Rare', 'Age'].values[0]
    return X_tmp

X_tmp = popuni_nedostajucu_dob(X_tmp)
    
# klasificiraj
klasificiraj(X_tmp)

Točnost:  0.814570707070707  +-  0.06399176749003999


### Inženjerstvo značajki
Inženjerstvo značajki je proces kojim se korištenjem znanja o nekoj domeni nastoje odabrati ili transformirati najbitnije varijable (značajke) iz pripremljenog skupa podataka s ciljem uspješnog modeliranja.  
Postupak koji smo ranije proveli za dobivanje titule putnika kao značajke mogao bi se smatrati inženjerstvom značajki. U nastavku ćemo pokazati još nekoliko primjera.  
  
Napravimo novu značajku koja će diskretizirati dob u pet kategorija - AgeBand

In [954]:
# popuni nedostajuću dob
X_tmp = popuni_nedostajucu_dob(X)

# stvorimo AgeBand
X_tmp['AgeBand'] = pd.cut(X_tmp['Age'], 5)
# prikažimo ovisnost s preživljavanjem
X_tmp[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

AgeBand  Survived
0    (0.34, 16.336]  0.548077
1  (16.336, 32.252]  0.392670
2  (32.252, 48.168]  0.317901
3  (48.168, 64.084]  0.426471
4    (64.084, 80.0]  0.090909

In [955]:
# primijenimo label enkoder za AgeBand
le = LabelEncoder()
le.fit(X.loc[:,'AgeBand'])

# transformiraj 
X.loc[:,'AgeBand'] = le.transform(X.loc[:,'AgeBand'])

Koristeći značajku AgeBand kreirat ćemo novu umjetnu značajku Age*Pclass

In [956]:
X['Age*Pclass'] = X.AgeBand * X.Pclass

X[['Age*Pclass', 'Survived']].groupby(['Age*Pclass'], as_index=False).mean().sort_values(by='Age*Pclass', ascending=True)

Age*Pclass  Survived
0           0  0.548077
1           1  0.733333
2           2  0.530055
3           3  0.323741
4           4  0.390625
5           6  0.139785
6           8  0.000000
7           9  0.111111
8          12  0.000000

Sada ćemo stvoriti novu značajku FamilySize koja će biti zbroj SibSp i Parch uvećano za 1 (uključujući tu osobu)

In [957]:
# stvorimo novu značajku
X['FamilySize'] = X['SibSp'] + X['Parch'] + 1

# ovisnost preživljavanja o novoj značajci
X[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='FamilySize', ascending=True)

FamilySize  Survived
0           1  0.300935
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000

Koristeći novu značajku FamilySize možemo kreirati još jednu novu značajku IsAlone.

In [958]:
# stvorimo novu značajku
X['IsAlone'] = 0
X.loc[X.loc[:,'FamilySize'] == 1, 'IsAlone'] = 1

# ovisnost preživljavanja o novoj značajci
X[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean().sort_values(by='IsAlone', ascending=True)

IsAlone  Survived
0        0  0.505650
1        1  0.300935

In [961]:
# izbaci nenumeričke stupce - ~ u ovom slučaju označava "not", odnosno uvjet bi se čitao kao "columns not in ['Name', ...]"
X_tmp = X.loc[:, ~X.columns.isin(['PassengerId', 'Name','Sex', 'Ticket', 'Cabin', 'Embarked', 'Title'])].copy()

# klasificiraj
klasificiraj(X_tmp)

Točnost:  0.8156818181818182  +-  0.06872742322532945


Provjerimo sada korelacije među značajkama. Napomena: prikaz matrice korelacija puno je bolji koristeći heatmap funkcije, no ovdje to nećemo koristiti jer je to tema sljedeće vježbe.

In [959]:
X.corr()

Survived    Pclass       Sex       Age     SibSp     Parch  \
Survived      1.000000 -0.335549 -0.541585 -0.094724 -0.034040  0.083151   
Pclass       -0.335549  1.000000  0.127741 -0.340679  0.081656  0.016824   
Sex          -0.541585  0.127741  1.000000  0.123741 -0.116348 -0.247508   
Age          -0.094724 -0.340679  0.123741  1.000000 -0.267239 -0.195976   
SibSp        -0.034040  0.081656 -0.116348 -0.267239  1.000000  0.414542   
Parch         0.083151  0.016824 -0.247508 -0.195976  0.414542  1.000000   
Fare          0.255290 -0.548193 -0.179958  0.088094  0.160887  0.217532   
Embarked     -0.169718  0.164681  0.110320 -0.026133  0.068900  0.040449   
Title_Master  0.085998  0.081547  0.159612 -0.412006  0.349434  0.267194   
Title_Miss    0.334953 -0.007761 -0.694744 -0.307473  0.084446  0.102026   
Title_Mr     -0.547689  0.139156  0.866888  0.237141 -0.252201 -0.335765   
Title_Mrs     0.340870 -0.151078 -0.550071  0.178464  0.063003  0.225519   
Title_Rare   -0.011611 -0.188273  0.075300  0.194984 -0.026055 -0.059725   
AgeBand      -0.114969 -0.303354  0.155596  0.930825 -0.254518 -0.200341   
Age*Pclass   -0.322618  0.378862  0.232564  0.607464 -0.239510 -0.180098   
FamilySize    0.018277  0.064221 -0.203191 -0.280584  0.890654  0.782988   
IsAlone      -0.206207  0.138553  0.306985  0.194706 -0.584186 -0.583112   

                  Fare  Embarked  Title_Master  Title_Miss  Title_Mr  \
Survived      0.255290 -0.169718      0.085998    0.334953 -0.547689   
Pclass       -0.548193  0.164681      0.081547   -0.007761  0.139156   
Sex          -0.179958  0.110320      0.159612   -0.694744  0.866888   
Age           0.088094 -0.026133     -0.412006   -0.307473  0.237141   
SibSp         0.160887  0.068900      0.349434    0.084446 -0.252201   
Parch         0.217532  0.040449      0.267194    0.102026 -0.335765   
Fare          1.000000 -0.226311      0.011390    0.118352 -0.181692   
Embarked     -0.226311  1.000000      0.031413   -0.096519  0.101336   
Title_Master  0.011390  0.031413      1.000000   -0.110890 -0.255888   
Title_Miss    0.118352 -0.096519     -0.110890    1.000000 -0.602266   
Title_Mr     -0.181692  0.101336     -0.255888   -0.602266  1.000000   
Title_Mrs     0.105511 -0.036499     -0.087798   -0.206644 -0.476850   
Title_Rare    0.016645 -0.029671     -0.035374   -0.083257 -0.192123   
AgeBand       0.080627 -0.049476     -0.371027   -0.315325  0.256970   
Age*Pclass   -0.276558  0.078341     -0.330774   -0.303750  0.361994   
FamilySize    0.218658  0.067305      0.372350    0.108698 -0.340148   
IsAlone      -0.274079  0.062532     -0.266840   -0.049521  0.399987   

              Title_Mrs  Title_Rare   AgeBand  Age*Pclass  FamilySize  \
Survived       0.340870   -0.011611 -0.114969   -0.322618    0.018277   
Pclass        -0.151078   -0.188273 -0.303354    0.378862    0.064221   
Sex           -0.550071    0.075300  0.155596    0.232564   -0.203191   
Age            0.178464    0.194984  0.930825    0.607464   -0.280584   
SibSp          0.063003   -0.026055 -0.254518   -0.239510    0.890654   
Parch          0.225519   -0.059725 -0.200341   -0.180098    0.782988   
Fare           0.105511    0.016645  0.080627   -0.276558    0.218658   
Embarked      -0.036499   -0.029671 -0.049476    0.078341    0.067305   
Title_Master  -0.087798   -0.035374 -0.371027   -0.330774    0.372350   
Title_Miss    -0.206644   -0.083257 -0.315325   -0.303750    0.108698   
Title_Mr      -0.476850   -0.192123  0.256970    0.361994   -0.340148   
Title_Mrs      1.000000   -0.065919  0.143532    0.038401    0.155742   
Title_Rare    -0.065919    1.000000  0.176388   -0.001859   -0.047650   
AgeBand        0.143532    0.176388  1.000000    0.705213   -0.274069   
Age*Pclass     0.038401   -0.001859  0.705213    1.000000   -0.253696   
FamilySize     0.155742   -0.047650 -0.274069   -0.253696    1.000000   
IsAlone       -0.365054    0.031245  0.201913    0.308512   -0.690654   

               IsAlone  
Survived

Vidimo kako je značajka FamilySize visoko korelirana s značajkama SibSp i Parch što navedene dvije značajke čini kandidatom za izbacivanje i smanjenje dimenzionalnosti skupa.  
  
Za sada ih nećemo izbacivati. Pokazat ćemo još filterske metode selekcije značajki.  
  
Prva metoda selekcije značajki je korištenje informacijske dobiti (mutual information). Funkcija mutual_info_classif svakoj značajki dodjeljuje njenu "važnost", a razred SelectKBest selektira k=7 najvažnijih značajki. Primjer je u nastavku.

In [972]:
kbest = SelectKBest(mutual_info_classif, k=7)

y_ = X_tmp.loc[:, 'Survived']
X_ = X_tmp.loc[:, X_tmp.columns != 'Survived']

kbest.fit(X_, y_)
X_.columns[kbest.get_support()].values

array(['Pclass', 'Age', 'Fare', 'Title_Mr', 'Title_Mrs', 'Age*Pclass',
       'FamilySize'], dtype=object)

In [969]:
columns = np.concatenate((X_.columns[kbest.get_support()].values, np.asarray(['Survived'])))
klasificiraj(X_tmp[columns])

Točnost:  0.8077020202020202  +-  0.05387444425988824


Odabrane su značajke 'Pclass', 'Age', 'Fare', 'Title_Mr', 'Title_Mrs', 'Age\*Pclass','FamilySize'. Točnost korištenjem samo tih značajki je nešto niža nego ranije, ali je model jednostavniji i vrijeme treniranja je kraće. U ovom primjeru to nije toliko značajno jer je broj značajki smanjen s 15 na 7. Na realnim projektima često postoji nekoliko tisuća značajki i nekoliko milijuna zapisa te se blago smanjenje točnosti smatra opravdanom cijenom za značajno smanjenje vremena treniranja.

Razred SelectKBest kao argument prima metodu koja rangira značajke u skupu podataka. Za tu svrhu može se koristiti i bilo koja vlastito definirana metoda. Sljedeći primjer pokazuje korištenje internog rangiranja značajki algoritma ExtraTreesClassifier za selekciju značajki.

In [973]:
def calc_extr_scores(X,y):
    cls = ExtraTreesClassifier()
    cls.fit(X,y)
    return cls.feature_importances_

kbest = SelectKBest(calc_extr_scores, k=7)
kbest.fit(X_, y_)
X_.columns[kbest.get_support()].values

array(['Pclass', 'Age', 'Fare', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
       'FamilySize'], dtype=object)

In [974]:
columns = np.concatenate((X_.columns[kbest.get_support()].values, np.asarray(['Survived'])))
klasificiraj(X_tmp[columns])

Točnost:  0.8202272727272726  +-  0.06397366814795333


U ovom slučaju smanjenje dimenzionalnosti dovelo je do malog povečanja točnosti. U odabranim značajkama postoji samo jedna razlika - umjesto 'Age\*Pclass' sada imamo 'Title_Miss'.